In [25]:
import pandas as pd 
import ast
from sklearn.metrics import f1_score
import unidecode

# Stage 1

In [26]:
polnear_only_fn = '../models_neural/quote_detection/output_file.txt'
with open(polnear_only_fn) as f:
    output = f.read()
    polnear_only_preds = list(filter(lambda x: x != '', output.split('\n')))
    polnear_only_preds = list(zip(polnear_only_preds[::2], polnear_only_preds[1::2]))
    polnear_only_preds = pd.Series(dict(polnear_only_preds))
    polnear_only_preds = polnear_only_preds.apply(lambda x: x.replace('{}', '')).apply(ast.literal_eval)

In [61]:
polnear_only_pred_df = (polnear_only_preds
 .apply(pd.Series)
 .unstack()
 .dropna()
 .reset_index()
 .rename(columns={'level_1': 'file_id', 'level_0': 'sent_idx', 0:'pred'})
 .sort_values(['file_id', 'sent_idx'])
 [['file_id', 'sent_idx', 'pred']]
 .reset_index(drop=True)
)

In [36]:
input_data_df = pd.read_csv('../data/our-annotated-data-full.csv.gz')

In [62]:
input_data_df

,label,sentence,entry_id,sent_idx,quote_type,tagline,source_type,affiliation,role,role_status
0,False,BANGKOK,doc_902,0,NaN,NaN,NaN,NaN,NaN,NaN
1,False,—,doc_902,1,NaN,NaN,NaN,NaN,NaN,NaN
2,True,A plane carrying key senior Laotian government...,doc_902,2,QUOTE,na,Named Group,Government,Informational,Current
3,False,Killed in the crash were two top figures in th...,doc_902,3,NaN,NaN,NaN,NaN,NaN,NaN
4,False,For a Communist party that relies on force and...,doc_902,4,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
6830,False,Scientists had expected an active hurricane se...,doc_730,10,NaN,NaN,NaN,NaN,NaN,NaN
6831,False,"That expectation has now been exceeded , along...",doc_730,11,NaN,NaN,NaN,NaN,NaN,NaN
6832,False,After exhausting the 21-name list that was est...,doc_730,12,NaN,NaN,NaN,NaN,NaN,NaN
6833,True,Mr. Feltgen said that “ 2020 had all the right...,doc_730,13,QUOTE,NaN,NaN,NaN,NaN,NaN


In [89]:
polnear_only_pred_and_label_df = (
    polnear_only_pred_df
    .merge(input_data_df, left_on=['file_id', 'sent_idx'], right_on=['entry_id', 'sent_idx'])
    .drop('entry_id', axis=1)
)

In [78]:
(
    polnear_only_pred_and_label_df
     .loc[lambda df: df['sentence'].str.strip().apply(unidecode.unidecode) != '"']
     .pipe(lambda df: f1_score(df['label'], df['pred']))
)

0.32312070986104136

In [ ]:
(
    polnear_only_pred_and_label_df
     .loc[lambda df: df['sentence'].str.strip().apply(unidecode.unidecode) != '"']
     .pipe(lambda df: f1_score(df['label'], df['pred']))
)

In [79]:
polnear_only_pred_and_label_df

,file_id,sent_idx,pred,label,sentence,quote_type,tagline,source_type,affiliation,role,role_status
0,doc_0,0,0.0,False,A two - day rally in global stocks looked set ...,NaN,NaN,NaN,NaN,NaN,NaN
1,doc_0,1,1.0,False,Tokyo stocks were down significantly at midday...,NaN,NaN,NaN,NaN,NaN,NaN
2,doc_0,2,1.0,False,Other markets in the Asian - Pacific region we...,NaN,NaN,NaN,NaN,NaN,NaN
3,doc_0,3,1.0,False,"U.S. Treasury bonds , typically seen by invest...",NaN,NaN,NaN,NaN,NaN,NaN
4,doc_0,4,0.0,False,Global stocks have been buoyed this week by pr...,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
6830,doc_956,51,0.0,True,"Gabriela Ramirez , 33 , said that “ there ’s s...",QUOTE,33,Named Individual,Witness,Participant,Current
6831,doc_956,52,0.0,True,“,QUOTE,NaN,NaN,NaN,NaN,NaN
6832,doc_956,53,1.0,True,What kind of society do you live in,QUOTE,NaN,NaN,NaN,NaN,NaN
6833,doc_956,54,0.0,True,", ” she said , “ if you just shoot with no con...",QUOTE,NaN,NaN,NaN,NaN,NaN
